<a href="https://colab.research.google.com/github/MSadatAnik/QuestionGeneration/blob/Sadat_Branch/QG_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#cell 1
!pip install transformers==4.48.0 datasets nltk rouge_score sacrebleu==2.4.3 meteor
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [1]:
#cell 2
import os
import json
import logging
import numpy as np
import torch
from itertools import chain
from dataclasses import dataclass
from typing import Optional
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq, HfArgumentParser,
    set_seed, AutoConfig, EarlyStoppingCallback,
)
from datasets import Dataset as HFDataset
from datasets import load_metric
from nltk.tokenize import word_tokenize

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
#cell 3
from google.colab import files

# Upload train and validation JSON files
print("Please upload qg_train_v0.json")
uploaded = files.upload()
print("Please upload qg_valid_v0.json")
uploaded = files.upload()

# Load JSON data
with open("qg_train_v0.json", "r") as f:
    train_data = json.load(f)

with open("qg_valid_v0.json", "r") as f:
    valid_data = json.load(f)

# Print dataset sizes
print("Train examples:", len(train_data))
print("Valid examples:", len(valid_data))

Please upload qg_train_v0.json


Saving qg_train_v0.json to qg_train_v0 (4).json
Please upload qg_valid_v0.json


Saving qg_valid_v0.json to qg_valid_v0 (4).json
Train examples: 192
Valid examples: 49


In [3]:
# Validate dataset and count questions
def validate_data(data, name):
    total_questions = 0
    invalid_entries = []
    normal_questions = 0
    cloze_questions = 0
    for i, item in enumerate(data):
        questions = item.get('questions', [])
        total_questions += len(questions)
        for q in questions:
            if not q.get('hl_context') or not q.get('question', {}).get('normal_format'):
                invalid_entries.append((i, q))
            if q.get('question', {}).get('normal_format'):
                normal_questions += 1
            if q.get('question', {}).get('cloze_format'):
                cloze_questions += 1
    if invalid_entries:
        logger.warning(f"Invalid entries in {name}: {invalid_entries}")
    else:
        logger.info(f"{name} dataset validated successfully.")
    logger.info(f"Total questions in {name}: {total_questions} (Normal: {normal_questions}, Cloze: {cloze_questions})")
    return total_questions, normal_questions, cloze_questions

train_total, train_normal, train_cloze = validate_data(train_data, "train")
valid_total, valid_normal, valid_cloze = validate_data(valid_data, "valid")

# Print sizes
print("Train items:", len(train_data))
print("Train questions:", train_total, f"(Normal: {train_normal}, Cloze: {train_cloze})")
print("Valid items:", len(valid_data))
print("Valid questions:", valid_total, f"(Normal: {valid_normal}, Cloze: {valid_cloze})")

Train items: 192
Train questions: 2726 (Normal: 2726, Cloze: 2726)
Valid items: 49
Valid questions: 671 (Normal: 671, Cloze: 671)


In [4]:
#cell 4
def tokenized_data(source_text, target_text, data_args, tokenizer):
    padding = "max_length" if data_args.pad_to_max_length else False
    model_inputs = tokenizer(source_text, max_length=data_args.max_source_length, padding=padding, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_text, max_length=data_args.max_target_length, padding=padding, truncation=True)
    if data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
#cell 5
class CustomDS(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data['input_ids'])
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.data.items()}

In [6]:
#cell 6
def process_qg_agno_openstax(data, data_args, tokenizer):
    all_questions = list(chain(*[d['questions'] for d in data]))
    source_text = []
    target_text = []
    for q in all_questions:
        if not q.get('question', {}).get('normal_format'):
            continue
        hl_context = q.get('hl_context', '').replace('<hl>', '').strip()
        if not hl_context:
            hl_context = "No context provided"
        inp_txt = f"context {hl_context}: "
        out_txt = q['question']['normal_format']
        source_text.append(inp_txt)
        target_text.append(out_txt)

    logger.info(f"Processed {len(source_text)} normal question-context pairs")
    if data_args.is_debug_mode > 0:
        source_text = source_text[:50]
        target_text = target_text[:50]

    model_inputs = tokenized_data(source_text, target_text, data_args, tokenizer)
    return CustomDS(model_inputs)


In [7]:
#cell 7
@dataclass
class DataTrainingArguments:
    task: str = "qg_agno"
    max_source_length: int = 512
    max_target_length: int = 48
    pad_to_max_length: bool = False
    ignore_pad_token_for_loss: bool = True
    is_debug_mode: int = -1

In [10]:
#cell 8
set_seed(42)
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

checkpoint_dir = "./results"
checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")] if os.path.exists(checkpoint_dir) else []
print("Available checkpoints:", checkpoints)

if checkpoints:
    checkpoint_path = os.path.join(checkpoint_dir, max(checkpoints, key=lambda x: int(x.split('-')[1])))
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
        print(f"Checkpoint {checkpoint_path} loaded successfully.")
    except Exception as e:
        print(f"Error loading checkpoint {checkpoint_path}: {e}")
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        print(f"Using base model {model_name} due to checkpoint error.")
else:
    print(f"No checkpoints found in {checkpoint_dir}, using base model {model_name}.")

Available checkpoints: []
No checkpoints found in ./results, using base model t5-small.


In [11]:
#cell 9
data_args = DataTrainingArguments()
train_dataset = process_qg_agno_openstax(train_data, data_args, tokenizer)
valid_dataset = process_qg_agno_openstax(valid_data, data_args, tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [20]:
#cell 10
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=6.25e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.1,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    warmup_steps=250,
    lr_scheduler_type="linear",
    predict_with_generate=True,
    push_to_hub=False,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="eval_rougeLsum",  # instead of eval_loss
    greater_is_better=True,
)

In [21]:
#cell 11

from datasets import load_metric
import sacrebleu
import re
from nltk.tokenize import word_tokenize

metric_rouge = load_metric("rouge")
metric_meteor = load_metric("meteor")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # 🔒 Fix OverflowError
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)

    # Decode
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    def normalize_text(text):
        text = text.lower().strip()
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s?]', '', text)
        return text

    # ROUGE
    rouge_result = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v.mid.fmeasure * 100, 4) for k, v in rouge_result.items()}

    # BLEU
    bleu_scores = [sacrebleu.corpus_bleu([pred], [[ref]]).score for pred, ref in zip(decoded_preds, decoded_labels)]
    result["bleu"] = round(np.mean(bleu_scores), 4)

    # METEOR
    meteor_result = metric_meteor.compute(predictions=decoded_preds, references=decoded_labels)
    result["meteor"] = round(meteor_result["meteor"] * 100, 4)

    # EM
    em_scores = [1 if normalize_text(pred) == normalize_text(ref) else 0 for pred, ref in zip(decoded_preds, decoded_labels)]
    result["em"] = round(np.mean(em_scores) * 100, 4)

    # F1
    def compute_f1(pred, ref):
        pred_tokens = set(word_tokenize(normalize_text(pred)))
        ref_tokens = set(word_tokenize(normalize_text(ref)))
        common = pred_tokens & ref_tokens
        if not pred_tokens or not ref_tokens:
            return 0.0
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(ref_tokens)
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)

    f1_scores = [compute_f1(pred, ref) for pred, ref in zip(decoded_preds, decoded_labels)]
    result["f1"] = round(np.mean(f1_scores) * 100, 4)

    return result


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [23]:
#cell 12
import shutil
shutil.rmtree("./results", ignore_errors=True)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=2,
        early_stopping_threshold=0.001
    )],
)

trainer.train()
trainer.save_model("./final_model")
print("Final model saved to ./final_model")

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Meteor,Em,F1
1,2.136800,2.224175,33.174200,12.923000,28.526400,28.506000,9.971800,26.515700,0.000000,38.247400
2,2.141800,2.221863,33.375700,13.198200,28.737600,28.704700,10.030700,26.831000,0.000000,38.489800
3,2.059900,2.226318,32.728900,12.530100,28.120800,28.055400,9.650200,26.079900,0.000000,37.850500
4,2.000800,2.221431,33.222200,13.106300,28.680000,28.641300,10.036200,26.663200,0.000000,38.213500


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Final model saved to ./final_model


In [27]:
#cell 13

import random
# Identify checkpoints
checkpoint_dir = "./results"
checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")] if os.path.exists(checkpoint_dir) else []
print("Available checkpoints:", checkpoints)

# Load the best available model
try:
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1])) if checkpoints else None
    model_path = os.path.join(checkpoint_dir, latest_checkpoint) if latest_checkpoint else "./final_model"

    if os.path.exists(os.path.join(model_path, "pytorch_model.bin")):
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
        print(f"Loaded {model_path} for evaluation.")
    else:
        raise FileNotFoundError("Checkpoint incomplete. Falling back.")
except Exception as e:
    print(f"Error loading {model_path}: {e}")
    model = AutoModelForSeq2SeqLM.from_pretrained("./final_model")
    print("Loaded final_model as fallback.")

# Build trainer again with model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Run evaluation
results = trainer.evaluate()
print("Evaluation Results:", results)

# Run prediction
raw_preds = trainer.predict(valid_dataset)
raw_pred_ids = np.clip(raw_preds.predictions, 0, tokenizer.vocab_size - 1)
decoded_preds = tokenizer.batch_decode(raw_pred_ids, skip_special_tokens=True)

# For context retrieval
def get_contexts_from_dataset(dataset):
    return [tokenizer.decode(inp, skip_special_tokens=True) for inp in dataset.data['input_ids']]

# Random sample of 5 predictions
indices = random.sample(range(len(decoded_preds)), 5)
contexts = get_contexts_from_dataset(valid_dataset)

print("\n📋 Sample Predictions with Contexts:")
for i, idx in enumerate(indices, 1):
    print(f"\n🔹 Sample {i}")
    print(f"📄 Context:\n{contexts[idx]}")
    print(f"❓ Predicted Question:\n{decoded_preds[idx]}")


Available checkpoints: ['checkpoint-172', 'checkpoint-344']
Error loading ./results/checkpoint-344: Checkpoint incomplete. Falling back.
Loaded final_model as fallback.


Evaluation Results: {'eval_loss': 2.22186279296875, 'eval_model_preparation_time': 0.0048, 'eval_rouge1': 33.3757, 'eval_rouge2': 13.1982, 'eval_rougeL': 28.7376, 'eval_rougeLsum': 28.7047, 'eval_bleu': 10.0307, 'eval_meteor': 26.831, 'eval_em': 0.0, 'eval_f1': 38.4898, 'eval_runtime': 33.5264, 'eval_samples_per_second': 20.014, 'eval_steps_per_second': 2.505}

📋 Sample Predictions with Contexts:

🔹 Sample 1
📄 Context:
context Myeloid stem cells give rise to all the other formed elements, including the erythrocytes ; megakaryocytes that produce platelets ; and a myeloblast lineage that gives rise to monocytes and three forms of granular leukocytes : neutrophils, eosinophils, and basophils.:
❓ Predicted Question:
Which of the following is not a myeloid stem cell?

🔹 Sample 2
📄 Context:
context Many dictatorships do not align themselves strictly with any particular belief system or ideology ; the goal of this type of regime is usually limited to preserving the authority of the dictator a